In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, grangercausalitytests, coint

In [2]:
def adf_test(timeseries):    
    test = adfuller(timeseries, autolag='AIC')
    result = pd.Series(test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in test[4].items() :
        result['Critical Value (%s)'%key] = value
    return result

In [3]:
def is_nonstationary(result):
    if abs(result['Test Statistic']) > abs(result['Critical Value (5%)']):
        return False
    else:
        return True

In [4]:
def print_adf_result(result):
    print ('Augmented Dickey-Fuller Unit Root Test\nNull Hypothesis : Time Series in Non-Stationary')
    print (result[['#Lags Used','Test Statistic','p-value','Critical Value (5%)']])
    print("At 5% significance level")
    if abs(result['Test Statistic']) > abs(result['Critical Value (5%)']):
        print ("Reject Null Hypothesis - Time Series is Stationary")
    else:
        print ("Failed to Reject Null Hypothesis - Time Series is Non-Stationary")

In [5]:
def print_coint_result(result):
    print("Engel Granger Cointegration Test\nNull Hypothesis : No Cointegration")
    print("t-Statistic :\t\t", result[0])
    print("p-value :\t\t", result[1])
    print("Critical Value 1% :\t", result[2][0])
    print("Critical Value 5% :\t", result[2][1])
    print("Critical Value 10% :\t", result[2][2])
    print("At 5% significance level")
    if(result[1]<=0.05):
        print("Reject Null Hypothesis - Cointegration is present")
    else:
        print("Failed to reject Null Hypothesis - No Cointegration")

In [6]:
def make_stationary(timeseries):
    diff_timeseries = timeseries.copy(deep=True)
    while(is_nonstationary(adf_test(diff_timeseries))):
        diff_timeseries = diff_timeseries.diff().dropna()
        
    return diff_timeseries

In [7]:
def print_granger_result(result):
    pvalues = []
    for i in result:
        pvalues.append(result[i][0]['params_ftest'][1])
    best_lag = pvalues.index(min(pvalues))+1
    print("Granger Causality")
    print("Best number of Lags :\t", best_lag)
    print("F-statistic :\t", result[best_lag][0]['params_ftest'][0])
    print("p-value :\t", result[best_lag][0]['params_ftest'][1])
    print("At 5% significance level")
    if(result[best_lag][0]['params_ftest'][1] < 0.05):
        print("Causality is significant")
    else:
        print("Causality is insignificant")

In [8]:
def run_tests(df):
    print("NIFTY")
    print_adf_result(adf_test(df['NIFTY']))
    print('\n')
    print("FOREX")
    print_adf_result(adf_test(df['FOREX']))
    print('\n')
    
    df['NIFTY_s'] = make_stationary(df['NIFTY'])
    df['FOREX_s'] = make_stationary(df['FOREX'])
    
    print("First Difference - NIFTY")
    print_adf_result(adf_test(df['NIFTY_s'].dropna()))
    print('\n')
    print("First Difference - FOREX")
    print_adf_result(adf_test(df['FOREX_s'].dropna()))
    print('\n')
    
    df=df.dropna()
    print("\n\nCointegration NIFTY-FOREX")
    coint_A = coint(df['NIFTY_s'], df['FOREX_s'], trend='c', maxlag=15)
    print_coint_result(coint_A)
    
    print("\nCausality from NIFTY to FOREX")
    granger_A = grangercausalitytests(df[['FOREX_s','NIFTY_s']].dropna(), maxlag=10, verbose=False)
    print_granger_result(granger_A)
    print("\nCausality from FOREX to NIFTY")
    granger_B = grangercausalitytests(df[['NIFTY_s','FOREX_s']].dropna(), maxlag=10, verbose=False)
    print_granger_result(granger_B)

In [9]:
niftydf = pd.read_csv('NIFTY.csv', index_col="Date")
forexdf = pd.read_csv('FOREX.csv', index_col="Date")
df = pd.concat([niftydf,forexdf], axis=1)
df['NIFTY'] = pd.to_numeric(df['NIFTY'], errors="coerce")
df['FOREX'] = pd.to_numeric(df['FOREX'], errors="coerce")
df = df.dropna()

In [10]:
i = df[df.index == '01-Jan-09'].iloc[0]
print(df.index.get_loc(i.name))
i = df[df.index == '31-Dec-14'].iloc[0]
print(df.index.get_loc(i.name))

1243
2718


In [11]:
df = df[1243:2719]
df

,NIFTY,FOREX
Date,,
01-Jan-09,2998.350,48.064999
02-Jan-09,3040.650,48.154999
05-Jan-09,3090.075,48.514499
06-Jan-09,3117.150,48.502500
07-Jan-09,3016.600,48.526998
...,...,...
24-Dec-14,8223.075,63.381501
26-Dec-14,8202.750,63.471500
29-Dec-14,8230.500,63.551500


In [ ]:
sys.stdout=open("result3.txt","w")
print("ANALYSIS FOR",df.index[0], ":", df.index[-1],"\n")
run_tests(df)
sys.stdout.close()